In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler
import polars as pl
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import  Bidirectional, GRU, Dense, Dropout,LSTM
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from tqdm import tqdm
import os

2024-11-25 16:20:54.872226: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
X_test = pd.read_csv("X_test_m4HAPAP.csv")
X_train = pd.read_csv("X_train_N1UvY30.csv")
y_train = pd.read_csv("y_train_or6m3Ta.csv")
num_classes = 24
Y= to_categorical(y_train['eqt_code_cat'], num_classes=num_classes)

In [ ]:
# Variables catégorielles à encoder
categorical_columns = ['venue', 'action', 'side']

# Variables numériques
numeric_columns = ['price', 'bid', 'ask', 'bid_size', 'ask_size', 'flux']

# Encoding
for col in categorical_columns:
    le = LabelEncoder()
    X_train[col] = le.fit_transform(X_train[col])
    X_test[col] = le.transform(X_test[col])


In [ ]:
def add_feature_engineering(df):
    df['bid_ask_ratio'] = df['bid_size'] / df['ask_size']
    df['Imbalance'] = df['bid_size'] - df['ask_size']

    df['price_change'] = df.groupby('obs_id')['price'].diff()
    df['cumulative_price_change'] = df.groupby('obs_id')['price_change'].cumsum()
    df['bid_change'] = df.groupby('obs_id')['bid'].diff()
    df['ask_change'] = df.groupby('obs_id')['ask'].diff()
    # VWAP (Volume-Weighted Average Price)
    # VWAP is calculated as the sum of (price * volume) divided by the total volume
    df['vwap'] = df.groupby('obs_id').apply(lambda g: (g['price'] * g['bid_size']).sum() / g['bid_size'].sum()).reindex(df.index)

    df['price_zscore'] = (df['price'] - df['price'].mean()) / df['price'].std()
    df['trade_volume'] = df['flux'] * df['trade'].astype(int)

     # 8. Application des transformations demandées
    df['log_bid_size'] = np.log(np.abs(df['bid_size'] + 1))
    df['log_ask_size'] = np.log(np.abs(df['ask_size'] + 1))
    df['log_flux'] = np.log(np.abs(df['flux']) + 1) * np.sign(df['flux'])

    return df

X_train = add_feature_engineering(X_train)
X_test = add_feature_engineering(X_test)


In [ ]:
selected_columns = [ 'venue', 'action', 'trade', 
                    'price',
                    'bid',
                    'ask',
                    'bid_size',
                    'ask_size',
                    'flux',
                    'Imbalance',
                    'bid_ask_ratio',
                    'cumulative_price_change',
                    'price_change',
                    'bid_change',
                    'ask_change',
                    'price_zscore',
                    'trade_volume',
                    'log_bid_size',
                    'log_ask_size',
                    'log_flux'] 

In [ ]:
def group_by_observation(X_train, selected_columns, sequence_length=100):
    grouped = X_train.groupby('obs_id')

    # Préallocation d'un tableau pour stocker toutes les séquences
    num_groups = len(grouped)
    sequences = np.zeros((num_groups, sequence_length, len(selected_columns)))  # Shape: (num_groups, sequence_length, num_features)

    for i, (_, group) in enumerate(grouped):
        # Prendre les sequence_length premiers événements si plus
        sequence = group[selected_columns].values[:sequence_length]

        # Si moins de sequence_length événements, on applique du padding (remplissage avec des zéros)
        if len(sequence) < sequence_length:
            sequences[i, :len(sequence)] = sequence  # Remplir les lignes avec les données existantes
            # Les lignes restantes sont déjà initialisées à zéro
        else:
            sequences[i] = sequence  # Remplacer par la séquence complète

    return sequences



# Utilisation de la fonction
X_train = group_by_observation(X_train, selected_columns)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Définir le modèle
class MyGRUModel(nn.Module):
    def __init__(self, num_unique_venues, num_unique_actions,num_unique_trade, embedding_dim, gru_units, num_classes):
        super(MyGRUModel, self).__init__()
        self.venue_embedding = nn.Embedding(num_unique_venues, embedding_dim)
        self.action_embedding = nn.Embedding(num_unique_actions, embedding_dim)
        self.gru = nn.GRU(input_size=embedding_dim * 2 + 16, hidden_size=gru_units, batch_first=True, bidirectional=True)
        self.fc1 = nn.Linear(gru_units * 2, 64)  # 2 car c'est bidirectionnel
        self.fc2 = nn.Linear(64, num_classes)

    def forward(self, x):
        venue_emb = self.venue_embedding(x[:, :, 0].long())
        action_emb = self.action_embedding(x[:, :, 1].long())
        trade_emb = self.action_embedding(x[:, :, 2].long())
        continuous_features = x[:, :, 3:]  # Assurez-vous que les dimensions correspondent
        features = torch.cat((venue_emb, action_emb,trade_emb, continuous_features), dim=-1)

        # Passer les données dans GRU
        gru_out, _ = self.gru(features)

        # On peut prendre la sortie de la dernière étape ou faire une autre opération ici
        output = self.fc1(gru_out[:, -1, :])
        output = torch.nn.functional.selu(output)
        output = self.fc2(output)
        return output


print ("class init done")
# Paramètres
embedding_dim = 8
gru_units = 64
num_classes = 24

# Instancier le modèle
model = MyGRUModel(num_unique_venues=6, num_unique_actions=3,num_unique_trade=2, embedding_dim=embedding_dim, 
                   gru_units=gru_units, num_classes=num_classes)

# Déplacer le modèle vers le GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Préparer vos données
inputs = torch.tensor(X_train, dtype=torch.float32).to(device)
targets = torch.tensor(Y, dtype=torch.float32).to(device)

# Créer DataLoader
dataset = TensorDataset(inputs, targets)
dataloader = DataLoader(dataset, batch_size=1000, shuffle=True)

# Configuration de l'optimiseur et de la fonction de perte
optimizer = optim.Adam(model.parameters(), lr=3e-3)
criterion = nn.CrossEntropyLoss()

print("model configuration = OK ")

In [ ]:
num_observations = X_train.shape[0]

In [ ]:

import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import torch.nn.functional as F

#STANDARD PARAMETERS#

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = .... 
# optimizer = torch.optim.Adam(model.parameters(), lr=5e-3)
# scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)


#LOSSES DEFINITIONS#

#MCC LOSS 



class MCCLoss(nn.Module):
    def __init__(self, T=20, epsilon=1e-5):
        super(MCCLoss, self).__init__()
        self.T = T
        self.epsilon = epsilon

    def entropy(self, input):
        """Compute the entropy for each batch element"""
        input = torch.clamp(input, min=self.epsilon)  # Éviter les log(0)
        entropy = -input * torch.log(input)
        return torch.sum(entropy, dim=1)

    def forward(self, targets):
        """
        Compute the MCC loss
        - targets: tensor of shape (batch_size, nb_class) with the model's logits
        """
        batch_size, nb_class = targets.size()

        # Rescale predictions with temperature
        rescaled_targets = F.log_softmax(targets / self.T, dim=1)

        # Vérifier si le rescaled_targets contient des NaN et les éviter
        if torch.isnan(rescaled_targets).any():
            print("NaN detected in softmax rescaled_targets.")
            rescaled_targets = torch.nan_to_num(rescaled_targets, nan=0.0)  # Remplacer NaN par 0

        # Compute entropy and calculate targets' weights
        targets_weights = self.entropy(rescaled_targets).detach()
        targets_weights = 1 + torch.exp(-targets_weights)
        targets_weights = batch_size * targets_weights / (torch.sum(targets_weights) + self.epsilon)  # Ajout d'epsilon pour éviter division par zéro

        # Compute covariance matrix
        cov_matrix = rescaled_targets.mul(targets_weights.view(-1, 1)).transpose(1, 0).mm(rescaled_targets)

        # Normaliser la matrice de covariance
        cov_matrix /= (torch.sum(cov_matrix, dim=1, keepdim=True) + self.epsilon)  # Ajout d'epsilon

        # Vérifier s'il y a des NaN dans la matrice de covariance et les remplacer
        if torch.isnan(cov_matrix).any():
            print("NaN detected in covariance matrix.")
            cov_matrix = torch.nan_to_num(cov_matrix, nan=0.0)

        # Compute MCC Loss
        loss = (torch.sum(cov_matrix) - torch.trace(cov_matrix)) / (nb_class + self.epsilon)  # Ajout d'epsilon dans la normalisation
        return loss


#LABEL SMOOTHING LOSS 
class LabelSmoothingLoss(nn.Module):
    def __init__(self, smoothing=0.1):
        super(LabelSmoothingLoss, self).__init__()
        self.smoothing = smoothing

    def forward(self, outputs, targets):
        targets = targets.long()  # S'assurer que c'est bien un entier
        num_classes = outputs.size(1)  # Nombre de classes

        # Convertir en one-hot encoding
        true_dist = torch.zeros_like(outputs).scatter_(1, targets, 1) 

        # Appliquer le lissage
        true_dist = true_dist * (1 - self.smoothing) + self.smoothing / num_classes

        # Calcul de la perte
        log_probs = nn.functional.log_softmax(outputs, dim=-1)
        loss = torch.mean(torch.sum(-true_dist * log_probs, dim=-1))

        return loss




#BIB:  Label Smoothing : Szegedy et al., "Rethinking the Inception Architecture for Computer Vision"



#TRAINING PLOT#

import matplotlib.pyplot as plt

def plot_training_progress(train_losses, val_losses, train_accuracies, val_accuracies, n=10):
    length = len(train_losses)
    epochs = range(1, length + 1)
    
    if length % n == 0:
        fig = plt.gcf()  # Get the current figure
        if fig is None:
            fig = plt.figure(figsize=(10, 6))  

        plt.clf()  # Clear the current figure
        
        # Loss Plot 
        plt.subplot(2, 1, 1)
        plt.plot(epochs, train_losses, label="Train Loss", color="blue", linestyle='-', marker='o')
        plt.plot(epochs, val_losses, label="Validation Loss", color="red", linestyle='--', marker='x')
        plt.xlabel('Epoch')
        
        if (max(train_losses)-min(train_losses))/(max(train_losses)+min(train_losses))>1/10:
            plt.yscale('log')
            plt.ylabel('Loss (Logscale)')
        else:
            plt.ylabel('Loss')
        plt.title(f'Train vs Validation Loss, epoch = {length}')
        plt.legend()
        
        # Accuracy Plot
        plt.subplot(2, 1, 2)
        plt.plot(epochs, train_accuracies, label="Train Accuracy", color="blue", linestyle='-', marker='o')
        plt.plot(epochs, val_accuracies, label="Validation Accuracy", color="red", linestyle='--', marker='x')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.title(f'Train vs Validation Accuracy, epoch = {length}')
        plt.legend()
        
        # Afficher la figure
        plt.tight_layout()
        plt.draw()
        plt.show()



# TRAINING #
import tqdm
import torch
import torch.nn as nn
import torch.optim as optim

import torch.nn.functional as F


def train(
    model,
    train_loader,
    val_loader,
    optimizer: torch.optim.Optimizer,
    scheduler: torch.optim.lr_scheduler._LRScheduler,
    num_epochs: int = 60,
    alpha: float = 0.1,
    beta: float = 0.1,
    device: str = "cuda" if torch.cuda.is_available() else "cpu",
    test_every: int = 10,
):
    """Entraîne le modèle avec Label Smoothing et MCC Loss."""

    # Définition des fonctions de perte
    criterion = nn.CrossEntropyLoss()
    label_smoothing_loss_fn = LabelSmoothingLoss()
    mcc_loss_fn = MCCLoss()

    # Envoi du modèle sur l'appareil cible (CPU/GPU)
    model.to(device)
    print(f"🔹 Training on device: {device}")

    best_val_accuracy = 0.0
    best_model_state = None

    # Suivi des métriques
    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []

    # Utilisation de `torch.cuda.amp` pour accélérer l'entraînement sur GPU
    scaler = torch.cuda.amp.GradScaler()

    # Boucle d'entraînement
    for epoch in range(1, num_epochs + 1):
        model.train()
        train_loss, train_accuracy, total_samples = 0.0, 0.0, 0
        mcc_loss_accumulated = 0.0  # Suivi de la MCC Loss

        for batch_idx, (inputs, targets) in enumerate(tqdm.tqdm(train_loader, desc=f"Epoch {epoch}/{num_epochs}", leave=False)):
            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()

            # Forward avec `amp`
            with torch.cuda.amp.autocast():
                outputs = model(inputs)

                # Perte principale
                loss =  criterion(outputs, targets.argmax(dim=1)) #+ alpha *label_smoothing_loss_fn(outputs, targets)

                # MCC Loss (ajoutée périodiquement)
                #if batch_idx % test_every == 0:
                    # mcc_loss = mcc_loss_fn(outputs)
                    # loss += beta * mcc_loss
                    #mcc_loss_accumulated += mcc_loss.item()

            # Backward avec `amp`
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            # Mise à jour des métriques
            print (loss.item)
            train_loss += loss.item()
            predicted = outputs.argmax(dim=1)
            targets_accs = targets.argmax(dim=1)
            train_accuracy += (predicted == targets_accs).sum().item()
            total_samples += targets.size(0)

        # Mise à jour du scheduler
        scheduler.step()

        # Moyenne des métriques
        train_loss /= len(train_loader)
        train_accuracy /= total_samples
        mcc_loss_accumulated /= max(1, len(train_loader) // test_every)

        print(f"📌 Epoch {epoch}: Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.4f}, MCC Loss: {mcc_loss_accumulated:.4f}")

        # Validation
        model.eval()
        val_loss, val_accuracy, val_samples = 0.0, 0.0, 0

        with torch.no_grad():
            for batch_idx, (inputs, targets) in enumerate(val_loader):
                inputs, targets = inputs.to(device), targets.to(device)

                outputs = model(inputs)

                loss = criterion(outputs, targets.argmax(dim=1)) #+ alpha *label_smoothing_loss_fn(outputs, targets)

                val_loss += loss.item()
                predicted = outputs.argmax(dim=1)
                targets_accs = targets.argmax(dim=1)
                val_accuracy += (predicted == targets_accs).sum().item()
                val_samples += targets.size(0)

                # MCC Loss (ajoutée périodiquement)
                if batch_idx % test_every == 0:
                    mcc_loss = mcc_loss_fn(outputs)
                    val_loss += beta * mcc_loss.item()

        val_loss /= len(val_loader)
        val_accuracy /= val_samples

        print(f"✅ Validation: Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}")

        # Stockage des métriques
        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)
        val_losses.append(val_loss)
        val_accuracies.append(val_accuracy)

        # Mise à jour du meilleur modèle
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            best_model_state = model.state_dict()
            
        plot_training_progress(train_losses, val_losses, train_accuracies, val_accuracies, n=3)
        for name, param in model.named_parameters():
            if param.grad is not None:
                print(f"{name}: {param.grad.abs().mean():.6f}")

    # Chargement du meilleur modèle
    model.load_state_dict(best_model_state)

    return model



In [ ]:
epoques = (10_000 * 1000) // num_observations
for epoch in range(epoques):  # Exemple avec 10 époques
    model.train()  # Mettre le modèle en mode entraînement
    for batch_inputs, batch_targets in dataloader:
        optimizer.zero_grad()  # Réinitialiser les gradients
        outputs = model(batch_inputs)  # Passer les données dans le modèle
        loss = criterion(outputs, batch_targets.argmax(dim=1))  # Calculer la perte
        loss.backward()  # Rétropropagation
        optimizer.step()  # Mise à jour des poids
    print(f'Epoch [{epoch+1}/{str(epoques)}], Loss: {loss.item():.4f}')


In [ ]:
X_test = add_feature_engineering(X_test)
X_test = group_by_observation(X_test, selected_columns)
X_test_input = torch.tensor(X_test, dtype=torch.float32).to(device)

In [ ]:
# Supposons que `test_dataset` contienne vos données de test
test_loader = DataLoader(X_test_input, batch_size=256, shuffle=False)

all_predictions = []
all_probabilities = []

model.eval()
with torch.no_grad():
    for batch_inputs in test_loader:  # Si vous avez besoin des cibles, sinon utilisez seulement batch_inputs
        batch_inputs = batch_inputs.to(device)

        # Faire la prédiction pour ce lot
        outputs = model(batch_inputs)
        probabilities = torch.softmax(outputs, dim=1)
        predicted_classes = torch.argmax(probabilities, dim=1)

        # Stocker les résultats
        all_predictions.append(predicted_classes.cpu().numpy())
        all_probabilities.append(probabilities.cpu().numpy())

# Combiner tous les lots
all_predictions = np.concatenate(all_predictions)
all_probabilities = np.concatenate(all_probabilities)

print("Prédictions pour tout le dataset :", all_predictions)


In [ ]:
df_prediction = pd.DataFrame(data = {'eqt_code_cat':all_predictions})
df_prediction
df_prediction.to_csv('ypredic.csv')